# Setup
## Libraries

In [42]:
import os, sys
import ee
import pandas as pd
import geopandas as gpd
import numpy as np
import xarray as xr
import xvec
import rioxarray as rxr
from dask.distributed import Client

sys.path.append('..')
from utils.constants import *

In [3]:
# Trigger the authentication flow.
ee.Authenticate()
# Initialize the library.
ee.Initialize(project=GEE_PROJECT_NAME)
ee.Initialize(opt_url='https://earthengine-highvolume.googleapis.com')
#4/1AQlEd8yYK3mNymn3Kfzu6ZuYTBalMaQMqLge9dTEhkv-a1kXKnROglHSNsc

In [4]:
client = Client(n_workers=2, threads_per_worker=2, memory_limit='16GB')
client

/maps/acz25/miniconda3/envs/nature-urban-geo-env/lib/python3.10/site-packages/distributed/node.py:187: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 34871 instead
  warnings.warn(


Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:34871/status,
Dashboard: http://127.0.0.1:34871/status,Workers: 2
Total threads: 4,Total memory: 29.80 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:36325,Workers: 2
Dashboard: http://127.0.0.1:34871/status,Total threads: 4
Started: Just now,Total memory: 29.80 GiB
Comm: tcp://127.0.0.1:44095,Total threads: 2
Dashboard: http://127.0.0.1:34287/status,Memory: 14.90 GiB
Nanny: tcp://127.0.0.1:33605,


## Directory and File Paths

In [5]:
life_data_path = RASTER_INPUT_DIR / "Eyres_et_al_LIFE/arable_0.25.tif"


# Data Processing
## Google Earth Engine Variables

In [6]:
# worldcover_ds = xr.open_dataset("ee://ESA/WorldCover/v100", engine='ee')
esri_land_ds = xr.open_dataset("ee://projects/sat-io/open-datasets/landcover/ESRI_Global-LULC_10m_TS", engine='ee', scale = .25).rio.set_spatial_dims(x_dim='lon', y_dim='lat')
ghs_built_s_ds = xr.open_dataset("ee://JRC/GHSL/P2023A/GHS_BUILT_S", engine='ee', scale = .25)
ghs_built_v_ds = xr.open_dataset("ee://JRC/GHSL/P2023A/GHS_BUILT_V", engine='ee', scale = .25)
ghs_smod_ds = xr.open_dataset("ee://JRC/GHSL/P2023A/GHS_SMOD", engine='ee', scale = .25)
ghs_pop_ds = xr.open_dataset("ee://JRC/GHSL/P2023A/GHS_POP", engine='ee', scale = .25)
alos_dsm_ds = xr.open_dataset("ee://JAXA/ALOS/AW3D30/V3_2", engine='ee', scale = .25)

In [59]:
def encode_smod_dataset(dataset, crs = None):
    ghs_smod_categories = {10.: 'water', 11.: 'very_low_density_rural', 12.: 'low_density_rural', 13.: 'rural_cluster', 
                        21.: 'suburban', 22.: 'semidense_urban', 23.: 'dense_urban', 30.: 'urban_centre', -200.: 'no_data'}

    one_hot_vars = []
    for k, v in ghs_smod_categories.items():
        one_hot = xr.where(ghs_smod_ds['smod_code'] == k, 1, 0)
        one_hot = one_hot.rename(f'smod_{v}')
        one_hot_vars.append(one_hot)

    res = xr.merge([ghs_smod_ds] + one_hot_vars).drop_vars('smod_code')
    res.attrs = dataset.attrs.copy()

    if crs:
        res = res.rio.write_crs(crs)
        
    return res

In [60]:
x = encode_smod_dataset(ghs_smod_ds)
x

<xarray.Dataset> Size: 896MB
Dimensions:                      (time: 12, lon: 1440, lat: 720)
Coordinates:
  * time                         (time) datetime64[ns] 96B 1975-01-01 ... 203...
  * lon                          (lon) float64 12kB -179.9 -179.6 ... 179.9
  * lat                          (lat) float64 6kB -89.88 -89.62 ... 89.62 89.88
Data variables:
    smod_water                   (time, lon, lat) int64 100MB 0 0 0 0 ... 0 0 0
    smod_very_low_density_rural  (time, lon, lat) int64 100MB 0 0 0 0 ... 0 0 0
    smod_low_density_rural       (time, lon, lat) int64 100MB 0 0 0 0 ... 0 0 0
    smod_rural_cluster           (time, lon, lat) int64 100MB 0 0 0 0 ... 0 0 0
    smod_suburban                (time, lon, lat) int64 100MB 0 0 0 0 ... 0 0 0
    smod_semidense_urban         (time, lon, lat) int64 100MB 0 0 0 0 ... 0 0 0
    smod_dense_urban             (time, lon, lat) int64 100MB 0 0 0 0 ... 0 0 0
    smod_urban_centre            (time, lon, lat) int64 100MB 0 0 0 0 ... 0 0 0
    smod_no_data                 (time, lon, lat) int64 100MB 0 0 0 0 ... 0 0 0
Attributes:
    crs:      EPSG:4326

In [63]:
merged_ds = xr.merge([ghs_built_s_ds, ghs_built_v_ds, ghs_smod_ds, ghs_pop_ds]).rio.set_spatial_dims(x_dim='lon', y_dim='lat')
merged_ds

<xarray.Dataset> Size: 299MB
Dimensions:             (time: 12, lon: 1440, lat: 720)
Coordinates:
  * time                (time) datetime64[ns] 96B 1975-01-01 ... 2030-01-01
  * lon                 (lon) float64 12kB -179.9 -179.6 -179.4 ... 179.6 179.9
  * lat                 (lat) float64 6kB -89.88 -89.62 -89.38 ... 89.62 89.88
Data variables:
    built_surface       (time, lon, lat) float32 50MB ...
    built_surface_nres  (time, lon, lat) float32 50MB ...
    built_volume_total  (time, lon, lat) float32 50MB ...
    built_volume_nres   (time, lon, lat) float32 50MB ...
    smod_code           (time, lon, lat) float32 50MB nan nan nan ... nan nan
    population_count    (time, lon, lat) float32 50MB ...
Attributes:
    crs:      EPSG:4326

## LIFE Variable

In [47]:
life_bands = {1: "amphibians", 2: "birds", 3: "mammals", 4: "reptiles"}
life_time = pd.date_range('2025-01-01', periods=1)
life_data_ds = rxr.open_rasterio(life_data_path).to_dataset(dim='band').rename(life_bands)\
    .rio.set_spatial_dims(x_dim='x', y_dim='y').expand_dims(time=life_time).rio.write_crs("EPSG:4326")
life_data_ds

<xarray.Dataset> Size: 4GB
Dimensions:      (time: 1, y: 10802, x: 21602)
Coordinates:
  * time         (time) datetime64[ns] 8B 2025-01-01
  * x            (x) float64 173kB -180.0 -180.0 -180.0 ... 180.0 180.0 180.0
  * y            (y) float64 86kB 90.01 89.99 89.97 ... -89.97 -89.99 -90.01
    spatial_ref  int64 8B 0
Data variables:
    amphibians   (time, y, x) float32 933MB nan nan nan nan ... nan nan nan nan
    birds        (time, y, x) float32 933MB nan nan nan nan ... nan nan nan nan
    mammals      (time, y, x) float32 933MB nan nan nan nan ... nan nan nan nan
    reptiles     (time, y, x) float32 933MB nan nan nan nan ... nan nan nan nan
Attributes:
    AREA_OR_POINT:       Area
    STATISTICS_MAXIMUM:  0.5256119966507
    STATISTICS_MEAN:     -9999
    STATISTICS_MINIMUM:  -0.08820016682148
    STATISTICS_STDDEV:   -9999
    _FillValue:          nan
    scale_factor:        1.0
    add_offset:          0.0
    long_name:           ('amphibians', 'birds', 'mammals', 'reptiles')

# Nigeria Example

In [10]:
# adm0_ds = xr.open_dataset("ee://WM/geoLab/geoBoundaries/600/ADM0", engine='ee')
gaul = ee.FeatureCollection("FAO/GAUL/2015/level0")
nigeria_boundary_gdf = ee.data.computeFeatures({
    'expression': gaul.filter(ee.Filter.eq('ADM0_NAME', 'Nigeria')),
    'fileFormat': 'GEOPANDAS_GEODATAFRAME'
})
nigeria_boundary_gdf.crs = gaul.first().geometry().projection().getInfo()['crs']
minx, miny, maxx, maxy = nigeria_boundary_gdf.total_bounds

In [64]:
nigeria_merged_ds = merged_ds.rio.clip_box(minx, miny, maxx, maxy)
nigeria_merged_ds = encode_smod_dataset(nigeria_merged_ds)